In [9]:
from SPARQLWrapper import SPARQLWrapper, JSON
from datetime import datetime
import pandas as pd
import numpy as np

connect to db

In [107]:
sparql = SPARQLWrapper('https://dati.camera.it/sparql')

Create dataframe of the deputati from a legislation

In [132]:
dep = pd.read_csv('data/18esima_Leg/dep_18.csv')[['persona', 'nomeGruppo', 'nome', 'cognome']]
a = dep['cognome']+ ' ' + dep['nome']
dep['Parlamentare'] = a
dep.drop(['nome', 'cognome'], axis = 1, inplace= True)

a = []
for i in range(dep.shape[0]):
    a.append(dep['persona'][i][39:])

dep['id'] = a
dep.drop('persona', inplace= True, axis = 1)

p = []
dat = []
for i in dep['nomeGruppo']:
    e = i.split()
    p.append(e[-2][1:-1])
    d = e[-1].replace(')', '')
    dat.append(datetime.strptime(d[-10:], '%d.%m.%Y'))

dep['Partito'] = p
dep['dataIngresso'] = dat
dep.drop('nomeGruppo', axis = 1, inplace= True)

final_df = pd.DataFrame(columns= dep.columns)

for i in np.unique(dep['id']):
    if len(dep[dep['id'] == i]) < 2:
        final_df = pd.concat([final_df, dep[dep['id'] == i]])
    else:
        duplicate = dep[dep['id'] == i]
        final_df = pd.concat([final_df, duplicate.head(1)])


In [127]:
year = '2022'

dates = []

month = '02'
for day in range(1,32):
    date = year+month+ f'{day:02d}'
    dates.append(date)

In [128]:
dates

['20220201',
 '20220202',
 '20220203',
 '20220204',
 '20220205',
 '20220206',
 '20220207',
 '20220208',
 '20220209',
 '20220210',
 '20220211',
 '20220212',
 '20220213',
 '20220214',
 '20220215',
 '20220216',
 '20220217',
 '20220218',
 '20220219',
 '20220220',
 '20220221',
 '20220222',
 '20220223',
 '20220224',
 '20220225',
 '20220226',
 '20220227',
 '20220228',
 '20220229',
 '20220230',
 '20220231']

In [ ]:
sparql.setReturnFormat(JSON)

leg = 18

# deputati of a specific legislation
for d in dates:
   sparql.setQuery("""
   select distinct ?votazione ?data ?titolo ?numeroVotazione ?cognome ?nome ?espressione ?infoAssenza ?deputato where {
   ## prima votazione del 8 febbraio 2022
   ?votazione a ocd:votazione;
      dc:date '""" + d + """';
      dc:date ?data;
      rdfs:label ?titolo;
      dc:description ?descrizione;
      dc:identifier ?numeroVotazione.


   ## voti espressi
   ?voto a ocd:voto;
      ocd:rif_votazione ?votazione;
      dc:type ?espressione;
      ocd:rif_deputato ?deputato.
   OPTIONAL{?voto dc:description ?infoAssenza}


   ## info deputato
   ?deputato foaf:surname ?cognome; foaf:firstName ?nome  


   } ORDER BY ?cognome
   
   """
   )
   try:
      ret = sparql.queryAndConvert()
      res = ret["results"]["bindings"]
   except Exception as e:
      print(e)

   dep = []
   vot = []
   numero = []
   for i in res:
      dep.append((i.get('deputato')).get('value')[40:46])
      vot.append((i.get('espressione')).get('value'))
      numero.append((i.get('numeroVotazione')).get('value'))

   votazione = pd.DataFrame([dep,vot, numero], index = ['id', 'Voto1', 'numero']).T
   
   for i in list(set(votazione['numero'])):
      espressione = votazione[votazione['numero'] == i][['id', 'Voto1']]
      espressione.columns= ('id', i)
      final_df = pd.merge(final_df, espressione, how = 'left')


In [134]:
final_df

,Parlamentare,id,Partito,dataIngresso,634001,635005,635022,635019,635017,635003,...,645012,645002,645005,645018,645024,645009,645025,645019,645007,645015
0,BASINI GIUSEPPE,200049,LEGA,2018-03-27,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato,...,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato
1,BERSANI PIER LUIGI,300026,MISTO,2018-03-23,Favorevole,Favorevole,Contrario,Favorevole,Favorevole,Favorevole,...,Contrario,Non ha votato,Contrario,Contrario,Contrario,Contrario,Contrario,Contrario,Contrario,Contrario
2,LETTA ENRICO,300127,PD,2018-03-27,Favorevole,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MINNITI DOMENICO,300146,PD,2018-03-27,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FRANCESCHINI DARIO,300246,PD,2018-03-27,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato,...,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato,Non ha votato
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,GIORGETTI GIANCARLO,50115,LEGA,2018-03-27,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
656,PAROLO UGO,50123,LEGA,2018-03-27,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
657,FOTI TOMMASO,50204,FDI,2018-03-27,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
658,RUSSO PAOLO,50213,FI,2018-03-27,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# se numero NA di un parlamentare è maggiore 

In [142]:
final_df.fillna('Assente', inplace= True)

In [144]:
final_df.columns[4:]

Index(['634001', '635005', '635022', '635019', '635017', '635003', '635002',
       '635011', '635012', '635015',
       ...
       '645012', '645002', '645005', '645018', '645024', '645009', '645025',
       '645019', '645007', '645015'],
      dtype='object', length=160)

In [145]:
dizionario_espressioni = {'Assente': '0', 'Favorevole': '1', 'Non ha votato': '0', 'Contrario': '-1', 'Astensione': '2', 'In missione': '0'}
for i in final_df.columns[4:]:
    final_df[i] = final_df[i].map(dizionario_espressioni)
    

In [147]:
from sklearn.metrics.pairwise import cosine_similarity

In [151]:
final_df.columns[4:]

Index(['634001', '635005', '635022', '635019', '635017', '635003', '635002',
       '635011', '635012', '635015',
       ...
       '645012', '645002', '645005', '645018', '645024', '645009', '645025',
       '645019', '645007', '645015'],
      dtype='object', length=160)